## `RNN`
***
***
Time: 2020-09-13<br>
Author: dsy
***

$
\begin{aligned}
z_t & = Uh_{t-1} + Wx_t+b \\
h_t & = f(z_t) \\
y_t & = Vh_t \\
x_t \text{是网络的输入}，h_t \text{隐藏层状态}，z_t \text{隐藏层的净输入}，f(\cdot) \text{
是非线性激活函数，通常为Logistic函数或者Tanh函数}，U \text{状态-状态权重矩阵},W \text{状态-输入权重矩阵},b\text{为偏置}
\end{aligned}
$

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class RNNFromDsy(nn.Module):
    def __init__(self,hidden_size=10):
        super(RNNFromDsy,self).__init__()
        self.hidden_size = hidden_size
        
    def forward(self,x):
        self.batch,self.input_size = x.shape
        ht = torch.zeros((self.batch,self.hidden_size))
        U = ((self.hidden_size,self.hidden_size))
        W = torch.empty(self.hidden_size,requires_grad=True)
        b = torch.empty(self.hidden_size,requires_grad=True)
        V = torch.empty(self.batch,self.batch,requires_grad=True)
        nn.init.uniform_(W)
        nn.init.uniform_(b)
        nn.init.uniform_(V)
        for i in range(self.hidden_size):
            ht = nn.functional.tanh(U * ht + W * x + b)
            print(V * ht)


In [ ]:
rnn = nn.RNN(input_size =128 , hidden_size =10,batch_first=True,bidirectional=False)
rnn

In [ ]:
rnnd = RNNFromDsy()
rnnd(torch.randn((65,128)))

In [ ]:
X = torch.randn((32,64,128)) # seq_len 32, batch 64, input_size 128
X.shape